# API

## План

- VK API
- OMDb API
- FastAPI: делаем API для своего сервиса

## VK API

VK API бесплатное, но нужно получить доступ. Документация: https://dev.vk.com/ru/reference

- Перейти на сайт ВК для разработчиков: https://dev.vk.com/ru, авторизоваться там, если попросят
- Создать новое приложение: тип Мини-приложение, название и категория любые
- Необходимо получить код подтверждения, может быть по номеру телефона или через пуш на телефон
- Если все ок, то вы перейдете в настройки приложения, там можно увидеть ключи доступа. На интересует сервисный ключ доступа (тут снова попросят код или пуш)

In [ ]:
import requests
from tqdm.auto import tqdm
from datetime import datetime
import pandas as pd

In [ ]:
TOKEN = ""
VERSION = "5.130"

### Что мы можем делать?

1. Выгрузить посты со страницы

Запрашиваем 2 последних поста со страницы юзера с id = 1 (Павел Дуров). Для сообществ ID будут отрицательными (например, -1).

In [ ]:
wall_get_url = "https://api.vk.com/method/wall.get"  # endpoint, на который мы отправляем такие запросы

In [ ]:
data = requests.get(
    wall_get_url,
    params={
        "owner_id": 1,  # ID юзера
        "count": 2,  # кол-во постов
        "v": VERSION, # версия API
        "access_token": TOKEN  # токен доступа
    }
).json()

Мы получим ответ, который представляет собой словарь, где по ключу `response` лежит сам ответ. Внутри лежит параметр `count` с числом записей (всего). В `items` сами посты (2, как мы просили). Для каждого поста есть информация по объекту `post`.

In [ ]:
data

{'response': {'count': 1020,
  'items': [{'inner_type': 'wall_wallpost',
    'donut': {'is_donut': False},
    'comments': {'count': 0},
    'marked_as_ads': 0,
    'hash': 'xTKRWTArhEygtZnIFQ',
    'type': 'post',
    'attachments': [],
    'date': 1525805964,
    'edited': 1525813826,
    'from_id': 1,
    'id': 2442097,
    'likes': {'can_like': 0, 'count': 236655, 'user_likes': 0},
    'owner_id': 1,
    'post_type': 'post',
    'reposts': {'count': 14227},
    'text': 'Иногда говорят, что Telegram был заблокирован в России, так как “закон есть закон”. Однако Telegram заблокирован в России как раз вопреки главному закону страны – Конституции. Решения судов и законы, противоречащие Конституции, не имеют силы. А это значит, что и сама блокировка Telegram незаконна. \n\nЕсли бы ФСБ ограничилась запросом информации о нескольких террористах, то ее требование вписывалось бы в рамки Конституции. Однако речь идет о передаче универсальных ключей шифрования с целью последующего бесконтрольно

Можно заметить, что дата отображается в виде числа. Это специальный формат unixtimestamp, который очень часто используется, так как целые числа - это универсальный способ хранения, который можно исопльзовать в любой системе (JSON, любые БД и прочие)

In [ ]:
unixtime = data['response']['items'][0]['date']
utc = datetime.fromtimestamp(unixtime)
print(unixtime, utc)

1525805964 2018-05-08 21:59:24


2. Выгрузить комментарии к постам

Основные параметры: `owner_id`, `post_id`. Их мы можем достать из информации о постах на стене. Можно это сделать и вручную, например, если открыть пост во всплывающем окне, то по адресу в адресной строке можно понять эти id.

https://vk.com/id1?w=wall1_2442097 : `owner_id = 1`, `post_id = 2442097`

Для примера возьмем пост из сообщества "Всего лишь писатель"

In [ ]:
get_comments_url = "https://api.vk.com/method/wall.getComments"  # здесь endpoint уже другой

In [ ]:
data = requests.get(
    get_comments_url,
    params={
        "owner_id": -142046107,
        "post_id": 1064437,
        "count": 2,
        "need_likes": 1,  # возвращать информацию о лайках
        "v": VERSION,
        "access_token": TOKEN
    }
).json()

In [ ]:
data

{'response': {'count': 45,
  'items': [{'id': 1064441,
    'from_id': 294374266,
    'date': 1733245976,
    'text': '',
    'post_id': 1064437,
    'owner_id': -142046107,
    'parents_stack': [],
    'attachments': [{'type': 'sticker',
      'sticker': {'inner_type': 'base_sticker_new',
       'sticker_id': 87603,
       'product_id': 1867,
       'images': [{'url': 'https://vk.com/sticker/1-87603-64',
         'width': 64,
         'height': 64},
        {'url': 'https://vk.com/sticker/1-87603-128',
         'width': 128,
         'height': 128},
        {'url': 'https://vk.com/sticker/1-87603-256',
         'width': 256,
         'height': 256},
        {'url': 'https://vk.com/sticker/1-87603-352',
         'width': 352,
         'height': 352},
        {'url': 'https://vk.com/sticker/1-87603-512',
         'width': 512,
         'height': 512}],
       'images_with_background': [{'url': 'https://vk.com/sticker/1-87603-64b',
         'width': 64,
         'height': 64},
        {'u

3. Выгрузить список пользователь в сообществе

In [ ]:
group_members = "https://api.vk.com/method/groups.getMembers"

group = "dormitory8hse"

In [ ]:
data = requests.get(
    group_members,
    params={
        'group_id': group,
        'access_token': TOKEN,
        'v': VERSION,
        'offset': 0
    }
).json()

data["response"]["count"]

6517

В качестве ответа по людям - просто список ID. По ним уже дальше можно запрашивать подробную информацию о пользователях.

In [ ]:
data["response"]["items"][:10]

[11952, 20090, 56613, 62028, 80420, 81206, 96206, 113393, 144980, 225392]

А теперь следующая страница. Это можно делать в цикле, чтобы выкачать всех

In [ ]:
data = requests.get(
    group_members,
    params={
        'group_id': group,
        'access_token': TOKEN,
        'v': VERSION,
        'offset': 1000  # смещение (начиная с 1000-го)
    }
).json()

data["response"]["items"][:10]

[26140160,
 26149926,
 26191419,
 26214484,
 26234658,
 26270770,
 26286469,
 26337548,
 26374113,
 26439214]

Можно делать еще много чего, но это лучше читать в документации :)

## OMDb API

Адрес сайта API: https://www.omdbapi.com/

Данный сервис является условно бесплатным: без доната доступно не больше 1000 запросов в день.

Как воспользоваться:
- Перейти на сайте во вкладку API Key и получить ключ (его еще надо будет активировать через почту)
- Отправлять запросы на адрес апи, используя полученный ключ

In [ ]:
API_KEY = ""
endpoint = f'http://www.omdbapi.com/'

In [ ]:
data = requests.get(
    endpoint,
    params={
        "t": "Harry Potter and the Chamber",
        "plot": "full",
        "apikey": API_KEY
    }
).json()
data

{'Title': 'Harry Potter and the Chamber of Secrets',
 'Year': '2002',
 'Rated': 'PG',
 'Released': '15 Nov 2002',
 'Runtime': '161 min',
 'Genre': 'Adventure, Family, Fantasy',
 'Director': 'Chris Columbus',
 'Writer': 'J.K. Rowling, Steve Kloves',
 'Actors': 'Daniel Radcliffe, Rupert Grint, Emma Watson',
 'Plot': 'Forced to spend his summer holidays with his muggle relations, Harry Potter (Daniel Radcliffe) gets a real shock when he gets a surprise visitor: Dobby (Toby Jones) the house-elf, who warns Harry against returning to Hogwarts, for terrible things are going to happen. Harry decides to ignore Dobby\'s warning and continues with his pre-arranged schedule. But at Hogwarts, strange and terrible things are indeed happening. Harry is suddenly hearing mysterious voices from inside the walls, muggle-born students are being attacked, and a message scrawled on the wall in blood puts everyone on his or her guard, "The Chamber Of Secrets Has Been Opened. Enemies Of The Heir, Beware".',
 

## FastAPI: делаем API для своего сервиса

Для быстрого создания своего API для сервиса на питоне мы можем использоваться библиотеку `FastAPI`, которая имеет много общего с Flask. Для работы с ней нам еще понадобиться библиотека `uvicorn`, которая отвечает за правильные запуск (можно попробовать без нее, но это рекомендованный способ от разработчиков библиотеки).

Также нам потребуется библиотека `pydantic`, которая отвечает за валидацию данных: мы не хотим руками прописывать все требования к входным данным (только тексты или только числа) и красивые реакции на ошибки в этих данных ("Извините, год $-$ это число, а не логическая переменная"), а данная библиотека это все умеет сама, если мы ей опишем правила, по которым проверять.

Попробуем сделать простое API, которое умеет делать три вещи:
- По запросу `info` возвращать список того, что она умеет
- По запросу `keywords` для переданного текста возвращать ключевые слова
- По запросу `normalize` для переданного текста возвращать его же очищенным от пунктуации и чисел и со всеми словами приведенными к начальной форме

К сожалению, код ниже не запустится в Юпитере из-за особенностей работы библиотеки (это жертва, на которую приходится идти ради возможности асинхронной работы), так что его надо скопировать в отдельный `.py` файл и запустить

Установим и импортируем все нужное

``` python
# !pip install fastapi uvicorn yake pydantic --q

from fastapi import FastAPI
import uvicorn
from pydantic import BaseModel

from pymorphy3 import MorphAnalyzer
from yake import KeywordExtractor
from nltk.tokenize import word_tokenize
```

Создадим все глобальные переменные и объекты

```python
morph = MorphAnalyzer()

max_ngram_size = 2
num_words = 5

language = 'ru'
custom_kw_extractor = KeywordExtractor(lan=language, n=max_ngram_size, top=num_words)
```

Опишем модель для входных данных. Здесь нам нужны только тексты, но если хочется чего-то сложного, то лучше подключить библиотеку `typing`.

```python
class TextQuery(BaseModel):
    text: str
```

Создадим приложение API и пропишем наши endpoint'ы

```python
app = FastAPI()


@app.get('/info')
def get_model_list() -> dict[str, list[str]]:
    return {'functions': ['keywords', 'normalize']}


# здесь делаем post, так как хотим передавать json в качестве параметра
@app.post('/keywords')
def query(params: TextQuery) -> dict[str, float]:
    keywords = custom_kw_extractor.extract_keywords(params.text)
    return {el[0]: el[1] for el in keywords}


@app.post('/normalize')
def query(params: TextQuery) -> dict[str, str]:
    text = [morph.parse(el)[0].normal_form \
                for el in word_tokenize(params.text) if el.isalpha()]
    return {'normalized_text': ' '.join(text)}
```

А вот так наше приложение можно запустить. Причем по адресу http://127.0.0.1:8000/docs#/ автоматически сгенерируется страница с документацией к нашему сервису и интерфейсом для тестирования.

```python
if __name__ == "__main__":
    uvicorn.run(app, port=8000, host="127.0.0.1")
```

Теперь давайте попробуем воспользоваться нашим API, как мы это делали с чужими

In [ ]:
base_url = 'http://127.0.0.1:8000'

In [ ]:
requests.get(base_url + '/info').json()

{'functions': ['keywords', 'normalize']}

In [ ]:
requests.post(
    base_url + '/keywords',
    json={'text': 'Привет! Как дела? Что делаешь?'}
).json()

{'Привет': 0.06588837669267192,
 'дела': 0.4949246952252326,
 'делаешь': 0.5880798524606783}

In [ ]:
requests.post(
    base_url + '/normalize',
    json={'text': 'Привет! Как дела? Что делаешь?'}
).json()

{'normalized_text': 'привет как дело что делать'}

А теперь посмотрим, что будет, если мы назовем поле неправильно

In [ ]:
requests.post(
    base_url + '/normalize',
    json={'text_data': 'Привет! Как дела? Что делаешь?'}
).json()

{'detail': [{'type': 'missing',
   'loc': ['body', 'text'],
   'msg': 'Field required',
   'input': {'text_data': 'Привет! Как дела? Что делаешь?'}}]}

Или положим в него не тот тип данных

In [ ]:
requests.post(
    base_url + '/normalize',
    json={'text': 10}
).json()

{'detail': [{'type': 'string_type',
   'loc': ['body', 'text'],
   'msg': 'Input should be a valid string',
   'input': 10}]}

Теперь у нас есть свое API и мы можем до бесконечности добавлять в него функции :)

## Задание

1.Добавьте к API выше следующие endpoint'ы:
- `get_user_posts`: должна принимать на вход id пользователя в ВК и кол-во постов, которые надо вернуть. А возвращать словарь с результатами (API всегда возвращают словарь!)
- `get_film_info`: должна принимать на вход id фильма и возвращать его название и сюжет
- `get_stat`: должна принимать на вход имя сообщества, название характеристики (возраст, пол) и число - кол-во людей, по которым считаем статистику. И возвращать частотную статистику (для числовых значений можно через гистограмму)

2. Сделайте так, чтобы endpoint'ы `keywords` и `normalize` умели работать со списками текстов